In [140]:
from sqlalchemy import create_engine
import pandas as pd


In [141]:
# === CONFIGURACIÓN DE CREDENCIALES ===
usuario = "AR_POWERBI"
contrasena = "Sim2024plit!"
host = "usasuxdb263p.cotyww.com"             # Ej: "localhost" o "192.168.1.100"
puerto = "1557"              # Puerto típico de Oracle
servicio = "INEVPRO1"     # Ej: "XEPDB1", "ORCL", etc.
# === CREAR EL ENGINE DE SQLAlchemy ===
dsn = f"{host}:{puerto}/{servicio}"
engine = create_engine(f"oracle+oracledb://{usuario}:{contrasena}@{dsn}")

# === TU CONSULTA ===
query = "SELECT CODIGO_GRUPO_ECONOMICO,DESCRIPCION FROM VW_GRUPO_ECONOMICO"  # Adaptá según lo que necesites

# === ABRIR UN DBAPI2 CONNECTION Y CARGAR EN DATAFRAME ===
conn = engine.raw_connection()
try:
    grupo_eco_simplit = pd.read_sql(query, con=conn)
    print("✅ Carga Exitosa")
finally:
    conn.close()

C:\Users\ArnaldoCarrasco\AppData\Local\Temp\ipykernel_30280\2722494975.py:17: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  grupo_eco_simplit = pd.read_sql(query, con=conn)


✅ Carga Exitosa


In [142]:
from unidecode import unidecode
import pandas as pd
import numpy as np
import os
from sqlalchemy import create_engine

#  Carga de Archivos Manuales

In [143]:
# path=r"C:\Users\ArnaldoCarrasco\Documents\SO SCRIPT\Consolidado\2. Carga SO Clientes\GRUPO ROUGE_CP_CS\7_202507.xlsx"
# df_combinado=pd.read_excel(path)
# df_combinado

In [144]:
# Ruta a la carpeta donde están las subcarpetas con los Excel
carpeta = r"C:\Users\ArnaldoCarrasco\Documents\SO SCRIPT\Consolidado\2. Carga SO Clientes"

# Lista para guardar todos los DataFrames
todos_los_dfs = []

# Recorrer todas las subcarpetas y archivos
for root, dirs, files in os.walk(carpeta):
    for archivo in files:
        if archivo.lower().endswith(('.xlsx', '.xls')):
            ruta_completa = os.path.join(root, archivo)
            try:
                df = pd.read_excel(ruta_completa)
                df['origen'] = os.path.splitext(archivo)[0]  # nombre del archivo sin extensión
                todos_los_dfs.append(df)
                print(f"✔️ Archivo leído: {archivo}")
            except Exception as e:
                print(f"⚠️ Error al leer {archivo}: {e}")

# Combinar los archivos si se encontró al menos uno
if todos_los_dfs:
    df_combinado = pd.concat(todos_los_dfs, ignore_index=True)
    print("✅ Archivos combinados exitosamente.")
    df_combinado.head()

    # Guardar el resultado a CSV o Excel si querés:
    # df_combinado.to_excel("consolidado_final.xlsx", index=False)
    # df_combinado.to_csv("consolidado_final.csv", index=False)

else:
    print("⚠️ No se encontraron archivos Excel válidos.")


✔️ Archivo leído: 1012_202507.xlsx
✔️ Archivo leído: 23_202507.xlsx
✔️ Archivo leído: 366_202506.xlsx
✔️ Archivo leído: 1706_202507.xlsx
✔️ Archivo leído: 2820_202507.xlsx
✔️ Archivo leído: 2031_202507.xlsx
✔️ Archivo leído: 1369_202507.xlsx
✔️ Archivo leído: 2690_202507.xlsx
✔️ Archivo leído: 42_202412.xlsx
✔️ Archivo leído: 42_202501.xlsx
✔️ Archivo leído: 42_202502.xlsx
✔️ Archivo leído: 42_202503.xlsx
✔️ Archivo leído: 42_202504.xlsx
✔️ Archivo leído: 42_202505.xlsx
✔️ Archivo leído: 42_202506.xlsx
✔️ Archivo leído: 42_202507.xlsx
✔️ Archivo leído: 2855_202507.xlsx
✔️ Archivo leído: 3340_202507.xlsx
✔️ Archivo leído: 3321_202507.xlsx
✔️ Archivo leído: 751_202507.xlsx
✔️ Archivo leído: 20_202507.xlsx
✔️ Archivo leído: 89_202507.xlsx
✔️ Archivo leído: 7_202507.xlsx
⚠️ Error al leer ~$7_202507.xlsx: [Errno 13] Permission denied: 'C:\\Users\\ArnaldoCarrasco\\Documents\\SO SCRIPT\\Consolidado\\2. Carga SO Clientes\\GRUPO ROUGE_CP_CS\\~$7_202507.xlsx'
✔️ Archivo leído: 267_202507.xlsx
✔️

In [145]:
# Asegurarse de que 'origen' sea string
df_combinado['origen'] = df_combinado['origen'].astype(str)

# Separar en dos columnas: 'grupo_economico' y 'periodo'
df_combinado[['grupo_economico', 'periodo']] = df_combinado['origen'].str.split('_', expand=True)

# Extraer año y mes correctamente
df_combinado['año'] = df_combinado['periodo'].str[:4]
df_combinado['mes'] = df_combinado['periodo'].str[4:]

# Crear columna 'fecha' como string "mm/yyyy"
df_combinado['fecha'] = df_combinado['mes'] + '/' + df_combinado['año']
# Crear columna 'fecha' como string en formato "01/mm/yyyy"
df_combinado['fecha'] = '01/' + df_combinado['mes'] + '/' + df_combinado['año']
df_combinado['fecha'] = pd.to_datetime(df_combinado['fecha'], format='%d/%m/%Y')

# Si querés la columna 'fecha' como datetime real:
# df_combinado['fecha'] = pd.to_datetime(df_combinado['año'] + '-' + df_combinado['mes'] + '-01')

# Verificamos el resultado
df_combinado.drop(columns=['mes', 'año','LRC' ,'periodo', 'Grupo economico', 'Fecha'], inplace=True)
df_combinado.rename(columns={
    'fecha': 'Fecha',
    'grupo_economico': 'Grupo id'
}, inplace=True)
grupo_eco_simplit['CODIGO_GRUPO_ECONOMICO'] = grupo_eco_simplit['CODIGO_GRUPO_ECONOMICO'].astype(str)
# Hacemos un merge trayendo solo la columna 'DESCRIPCION'
df_combinado = df_combinado.merge(
    grupo_eco_simplit[['CODIGO_GRUPO_ECONOMICO', 'DESCRIPCION']],
    how='left',
    left_on='Grupo id',
    right_on='CODIGO_GRUPO_ECONOMICO'
)

# Opcional: eliminar la columna 'CODIGO_GRUPO_ECONOMICO' si no la necesitás
df_combinado.drop(columns=['CODIGO_GRUPO_ECONOMICO'], inplace=True)

# Renombrar 'DESCRIPCION' si querés un nombre más claro
# df_combinado.rename(columns={'DESCRIPCION': 'nombre_grupo'}, inplace=True)
df_combinado.drop(columns=['Grupo id'], inplace=True)
df_combinado.rename(columns={
    'DESCRIPCION':'Grupo economico' 
}, inplace=True)
# Verificamos el resultado

In [146]:
df_combinado

,Ean,Desc Prod,Marca,Punto de Venta,Cantidad,Importe Neto,Stock,origen,Tipo de venta,Fecha,Grupo economico
0,3.616305e+12,ADIDAS CHILL ZONE EDP 100,NaN,NaN,0.0,NaN,22.0,1012_202507,NaN,2025-07-01,ASOPROFARMA C.P.L.
1,3.616305e+12,ADIDAS ENERGY DRI EDP 100,NaN,NaN,3.0,NaN,11.0,1012_202507,NaN,2025-07-01,ASOPROFARMA C.P.L.
2,3.616305e+12,ADIDAS FULL RECHAR EDP100,NaN,NaN,1.0,NaN,17.0,1012_202507,NaN,2025-07-01,ASOPROFARMA C.P.L.
3,3.616305e+12,ADIDAS GET COMFY EDP 100,NaN,NaN,1.0,NaN,16.0,1012_202507,NaN,2025-07-01,ASOPROFARMA C.P.L.
4,3.616305e+12,ADIDAS HAPPY FEELS EDP100,NaN,NaN,2.0,NaN,23.0,1012_202507,NaN,2025-07-01,ASOPROFARMA C.P.L.
...,...,...,...,...,...,...,...,...,...,...,...
483172,3.616306e+12,RIM- BLUR IT OUT KIND & FREE,NaN,NaN,1.0,13801.0,NaN,22_202507,NaN,2025-07-01,TIENDA LOS GALLEGOS
483173,3.616304e+12,RIM- KIND & FREE CLEAN BROWN WAX,NaN,NaN,1.0,7272.0,NaN,22_202507,NaN,2025-07-01,TIENDA LOS GALLEGOS
483174,3.616304e+12,RIM- KIND & FREE CLEAN BROWN WAX,NaN,NaN,1.0,7272.0,NaN,22_202507,NaN,2025-07-01,TIENDA LOS GALLEGOS
483175,3.616304e+12,RIM- KIND & FREE CLEAN BROWN WAX,NaN,NaN,1.0,7272.0,NaN,22_202507,NaN,2025-07-01,TIENDA LOS GALLEGOS


# Origenes

In [147]:
# Ruta del archivo de Excel
# file_path = r"C:\Users\ArnaldoCarrasco\Documents\SO SCRIPT\00.1 SO Enero\SOJunio.xlsx"
file_path2 = r"C:\Users\ArnaldoCarrasco\Documents\SO SCRIPT\00.1 SO Enero\EAN_Mapeo.xlsx"
file_path3 = r"C:\Users\ArnaldoCarrasco\Documents\SO SCRIPT\00.0 Dimensionales\GPS.xlsx"
file_path4 = r'C:\Users\ArnaldoCarrasco\Documents\SO SCRIPT\00.0 Dimensionales\Buscador EAN.xlsx'
# Cargar el archivo de Excel en un DataFrame
df = df_combinado
ean_mapeo= pd.read_excel(file_path2)
gps= pd.read_excel(file_path3)
ean= pd.read_excel(file_path4)
gps_cliente= pd.read_excel(file_path3,sheet_name='CLIENTES')

In [148]:
# ean_mapeo= pd.read_excel(file_path2)
ean_mapeo['Ean'].fillna(0, inplace=True)
ean_mapeo['Ean']=ean_mapeo['Ean'].astype(np.int64)
ean_mapeo['Ean Correcto'].fillna(0, inplace=True)
ean_mapeo['Ean Correcto']=ean_mapeo['Ean Correcto'].astype(np.int64)
# ean_mapeo.info()

In [149]:
df['Stock'].fillna(0, inplace=True)
df['Stock']=df['Stock'].astype(np.int64)
df['Importe Neto'].fillna(0, inplace=True)
df['Importe Neto']=df['Importe Neto'].astype(np.float64)
df['Ean'] = df['Ean'].astype(str).str.split('.').str[0]
# Filtrar los que contienen algún carácter no numérico
ean_invalidos = df[~df['Ean'].str.match(r'^\d+$')]

ean_invalidos

,Ean,Desc Prod,Marca,Punto de Venta,Cantidad,Importe Neto,Stock,origen,Tipo de venta,Fecha,Grupo economico
458148,nan,Tiffany & Co. Intense Rose Gold EDP + Ti,TIFFANY & CO,WEB,1.0,203840.00,0,13_202507,NaN,2025-07-01,JULERIAQUE
458149,nan,Tiffany & Co. Intense Rose Gold EDP + Ti,TIFFANY & CO,WEB,1.0,203840.00,0,13_202507,NaN,2025-07-01,JULERIAQUE
458150,nan,Tiffany & Co. Intense Rose Gold EDP + Ti,TIFFANY & CO,WEB,1.0,203840.00,0,13_202507,NaN,2025-07-01,JULERIAQUE
458151,nan,Tiffany & Co. Intense Rose Gold EDP + Ti,TIFFANY & CO,WEB,1.0,204871.16,0,13_202507,NaN,2025-07-01,JULERIAQUE
458152,nan,Tiffany & Co. Intense Rose Gold EDP + Ti,TIFFANY & CO,WEB,1.0,203840.00,0,13_202507,NaN,2025-07-01,JULERIAQUE
...,...,...,...,...,...,...,...,...,...,...,...
461705,nan,Boss Bottled Elixir Parfum 100 ml + Week,HUGO BOSS,WEB,1.0,176400.00,0,13_202507,NaN,2025-07-01,JULERIAQUE
461706,nan,Boss Bottled Elixir Parfum 100 ml + Week,HUGO BOSS,WEB,1.0,176400.00,0,13_202507,NaN,2025-07-01,JULERIAQUE
461707,nan,Boss Bottled Elixir Parfum 100 ml + Week,HUGO BOSS,WEB,1.0,176400.00,0,13_202507,NaN,2025-07-01,JULERIAQUE
461708,nan,Boss Bottled Elixir Parfum 100 ml + Week,HUGO BOSS,WEB,1.0,176400.00,0,13_202507,NaN,2025-07-01,JULERIAQUE


In [150]:
df['Ean'].replace('nan', 0, inplace=True)
df['Ean'] = df['Ean'].astype(np.int64)

df_historico=df.copy()
df_historico.loc[df["Fecha"].isna(), "Fecha"] = pd.to_datetime("1/6/2025", format="%d/%m/%Y")
df_historico["Fecha"] = pd.to_datetime(df_historico["Fecha"], errors='coerce', dayfirst=True)
df_historico["Fecha"] = pd.to_datetime(df_historico["Fecha"], format="%d/%m/%Y")
df_historico['Ean'] = df_historico['Ean'].astype(str).str.split('.').str[0]

# df.info()


In [151]:
gps['Cliente'].fillna(0, inplace=True)
gps['Cliente']=gps['Cliente'].astype(str)
gps_cliente['ID_CLIENTE']=gps_cliente['ID_CLIENTE'].astype(str)

gps = gps.merge(
    gps_cliente[['ID_CLIENTE', 'Nombre Correcto']],
    how='left',
    left_on='Cliente',
    right_on='ID_CLIENTE'
)


# Carga de Cambios

Correciones de eans erroneos en base al archivo historico de correcciones

In [152]:
ean_historico = ean_mapeo[(ean_mapeo['Ean Correcto'] > 0) | ((ean_mapeo['Ean Correcto'] == 0) & (ean_mapeo['Tester'] == 'Si'))]
ean_historico_errors = ean_mapeo[(ean_mapeo['Ean Correcto'] < 0) | ((ean_mapeo['Ean Correcto'] != 0) & (ean_mapeo['Tester'] == 'Si'))]
ean_historico_unico = ean_historico.drop_duplicates(subset=['Desc Prod'], keep='first')
df_corregido = pd.merge(
    df,
    ean_historico_unico[['Desc Prod', 'Ean Correcto', 'Tester']],
    on='Desc Prod',
    how='left'
)
df_corregido['Ean Correcto'].fillna(0, inplace=True)
df_corregido['Ean Correcto'] = df_corregido['Ean Correcto'].astype(np.int64)
df_corregido = df_corregido[df_corregido['Tester'] != 'Si']
df_corregido.loc[df_corregido['Ean Correcto'] > 0, 'Ean'] = df_corregido.loc[df_corregido['Ean Correcto'] > 0, 'Ean Correcto']
df_corregido = df_corregido.copy()
df_corregido.drop(columns=['Ean Correcto', 'Tester'], inplace=True)
df=df_corregido.copy()

# Transformaciones

In [153]:
df["Tipo de Venta"]="Sell out"
# df.loc[df["Fecha"].isna(), "Fecha"] = pd.to_datetime("1/6/2025", format="%d/%m/%Y")
# df["Fecha"] = pd.to_datetime(df["Fecha"], errors='coerce', dayfirst=True)
# df["Fecha"] = pd.to_datetime(df["Fecha"], format="%d/%m/%Y")
df['Ean'] = df['Ean'].fillna(0)
df["Ean sist"] = df["Ean"].isin(ean["EAN"])
# df["Ean LRC"] = df["LRC"].isin(ean["LRC"])

## Mapeo GPS

Eliminamos los puntos de ventas repetidos que pudiesen existir

In [154]:
# gps_duplicates = gps.duplicated(subset=["Codigo cliente", "Cliente"], keep=False)

In [155]:
# Normalizar una columna específica (minúsculas y sin acentos)
def normalizar_columna(col):
    col.astype(str)
    return col.str.lower().apply(unidecode).str.strip()

gps_unico = gps.drop_duplicates(subset=["Codigo cliente", "Cliente"], keep='first').copy()
# Convertir columnas de df_combinado
df['Punto de Venta'] = df['Punto de Venta'].astype(str)
df['Grupo economico'] = df['Grupo economico'].astype(str)

# Convertir columnas del otro DataFrame
gps_unico['Codigo cliente'] = gps_unico['Codigo cliente'].astype(str)

df_merged = pd.merge(
    df,
    gps_unico[["Código Coty","Codigo cliente","Nombre Correcto","Ecommerce"]],
    left_on=["Punto de Venta","Grupo economico"],
    right_on=["Codigo cliente","Nombre Correcto"],
    how='left',
    indicator=True
)

### Configuracion para PDV MIXTOS (Farmacity)

In [156]:
df_merged['Ecommerce'] = np.where(df_merged['Ecommerce'] == 'MIXTO', df_merged['Tipo de venta'], df_merged['Ecommerce'])


In [157]:
df_merged['Ecommerce'] = np.where(df_merged['Ecommerce'] == 'OFFLINE', False, np.where(df_merged['Ecommerce'] == 'ONLINE', True, df_merged['Ecommerce']))

In [158]:
# df_merged2 = pd.merge(
#     df_merged,
#     gps_cliente[["ID_CLIENTE","Nombre Correcto"]],
#     left_on=["Grupo economico"],
#     right_on=["ID_CLIENTE"],
#     how='left'
#     # indicator=True
# )

In [159]:
# Lista de columnas a eliminar
# columnas_a_eliminar = ["Cliente","ID_CLIENTE"]

# Eliminar columnas del DataFrame
# df_merged2 = df_merged2.drop(columns=columnas_a_eliminar)
df=df_merged
df = df.drop('Tipo de venta', axis=1)
df = df.rename(columns={"Nombre Correcto": "Grupo economico Final"})
df = df.rename(columns={"Código Coty": "PDV GPS"})
df["Punto de Venta"] = df["Punto de Venta"].astype(str)
gps["Codigo cliente"] = gps["Codigo cliente"].astype(str)
df['Importe Neto'] = df['Importe Neto'].astype(float)
df

,Ean,Desc Prod,Marca,Punto de Venta,Cantidad,Importe Neto,Stock,origen,Fecha,Grupo economico,Tipo de Venta,Ean sist,PDV GPS,Codigo cliente,Grupo economico Final,Ecommerce,_merge
0,3616305447197,ADIDAS CHILL ZONE EDP 100,NaN,nan,0.0,0.0,22,1012_202507,2025-07-01,ASOPROFARMA C.P.L.,Sell out,True,NaN,NaN,NaN,NaN,left_only
1,3616305447067,ADIDAS ENERGY DRI EDP 100,NaN,nan,3.0,0.0,11,1012_202507,2025-07-01,ASOPROFARMA C.P.L.,Sell out,True,NaN,NaN,NaN,NaN,left_only
2,3616305447050,ADIDAS FULL RECHAR EDP100,NaN,nan,1.0,0.0,17,1012_202507,2025-07-01,ASOPROFARMA C.P.L.,Sell out,True,NaN,NaN,NaN,NaN,left_only
3,3616305447166,ADIDAS GET COMFY EDP 100,NaN,nan,1.0,0.0,16,1012_202507,2025-07-01,ASOPROFARMA C.P.L.,Sell out,True,NaN,NaN,NaN,NaN,left_only
4,3616305447159,ADIDAS HAPPY FEELS EDP100,NaN,nan,2.0,0.0,23,1012_202507,2025-07-01,ASOPROFARMA C.P.L.,Sell out,True,NaN,NaN,NaN,NaN,left_only
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
484214,3616306361805,RIM- BLUR IT OUT KIND & FREE,NaN,nan,1.0,13801.0,0,22_202507,2025-07-01,TIENDA LOS GALLEGOS,Sell out,True,NaN,NaN,NaN,NaN,left_only
484215,3616303995683,RIM- KIND & FREE CLEAN BROWN WAX,NaN,nan,1.0,7272.0,0,22_202507,2025-07-01,TIENDA LOS GALLEGOS,Sell out,True,NaN,NaN,NaN,NaN,left_only
484216,3616303995683,RIM- KIND & FREE CLEAN BROWN WAX,NaN,nan,1.0,7272.0,0,22_202507,2025-07-01,TIENDA LOS GALLEGOS,Sell out,True,NaN,NaN,NaN,NaN,left_only
484217,3616303995683,RIM- KIND & FREE CLEAN BROWN WAX,NaN,nan,1.0,7272.0,0,22_202507,2025-07-01,TIENDA LOS GALLEGOS,Sell out,True,NaN,NaN,NaN,NaN,left_only


# Transformaciones Particulares para Clientes Específicos

## Restar Iva al importe

In [160]:
df.loc[df['Grupo economico'] == 'FREE SHOP S.A.', 'Importe Neto'] = df['Importe Neto'] / 1.21
df.loc[df['Grupo economico'] == 'GRUPO JUFEC', 'Importe Neto'] = df['Importe Neto'] / 1.21
df.loc[df['Grupo economico'] == 'PERFUGROUP', 'Importe Neto'] = df['Importe Neto'] / 1.21
df.loc[df['Grupo economico'] == 'FARMACIA DEL PUEBLO S.C.S.', 'Importe Neto'] = df['Importe Neto'] / 1.21
df.loc[df['Grupo economico'] == 'JULERIAQUE', 'Importe Neto'] = df['Importe Neto'] / 1.21

## Completar PDV con Grupo

In [161]:
df.loc[df['Grupo economico'] == 'DROGUERIA 20 DE JUNIO S.A.', ['PDV GPS', 'Ecommerce']] = ['DROGUERIA 20 DE JUNIO S.A.',False]
# df.loc[df['Grupo economico Final'] == 'DROGUERIA 20 DE JUNIO S.A.', 'PDV GPS'] = 'DROGUERIA 20 DE JUNIO S.A.'
df.loc[df['Grupo economico'] == 'COFARSUR S.A.', ['PDV GPS', 'Ecommerce']] = ['COFARSUR S.A.',False]
df.loc[df['Grupo economico'] == 'FIRST LABEL SRL', ['PDV GPS', 'Ecommerce']] = ['FIRST LABEL SRL.', True]
df.loc[df['Grupo economico'] == 'NIPPON', ['PDV GPS', 'Ecommerce']] = ['Nippon Nippon', False]
df.loc[df['Grupo economico'] == 'DROGUERIA SUR S.A.', ['PDV GPS', 'Ecommerce']] = ['DROGUERIA SUR S.A.', False]
# df.loc[df['Grupo economico Final'] == 'DROGUERIA SUIZO', ['PDV GPS', 'Ecommerce']] = ['DROGUERIA SUIZO', False]
df.loc[df['Grupo economico'] == 'FARMAX SRL', ['PDV GPS', 'Ecommerce']] = ['FARMAX SRL', False]
df.loc[df['Grupo economico'] == 'TIENDA LOS GALLEGOS', ['PDV GPS', 'Ecommerce']] = ['TIENDA LOS GALLEGOS', False]
df.loc[df['Grupo economico'] == 'PRODEFARM S.A.', ['PDV GPS', 'Ecommerce']] = ['PRODEFARM S.A.', False]
df.loc[df['Grupo economico'] == 'PERFUMERIAS ALDAZ', ['PDV GPS', 'Ecommerce']] = ['PERFUMERIAS ALDAZ', False]
df.loc[df['Grupo economico'] == 'CASTRO JOSE MANUEL Y OTRO SOC.DE HECHO', ['PDV GPS', 'Ecommerce']] = ['CASTRO JOSE MANUEL Y OTRO SOC.DE HECHO', False]
df.loc[df['Grupo economico'] == 'KELLER', ['PDV GPS', 'Ecommerce']] = ['KELLER', False]
df.loc[df['Grupo economico'] == 'TIENDA LOS GALLEGOS', ['PDV GPS', 'Ecommerce']] = ['TIENDA LOS GALLEGOS', False]
df.loc[df['Grupo economico'] == 'MARIA UÑA DE CARLETTO', ['PDV GPS', 'Ecommerce']] = ['MAGLIONE', False]
df.loc[df['Grupo economico'] == 'COOP.FARMACEUTICA MENDOZA LTDA.', ['PDV GPS', 'Ecommerce']] = ['COOP.FARMACEUTICA MENDOZA LTDA.', False]
df.loc[df['Grupo economico'] == 'ASOPROFARMA C.P.L.', ['PDV GPS', 'Ecommerce']] = ['ASOPROFARMA C.P.L.', False]
df.loc[df['Grupo economico'] == 'ADM FRAGANCIAS S.R.L.', ['PDV GPS', 'Ecommerce']] = ['ADM FRAGANCIAS S.R.L.', False]
df.loc[df['Grupo economico'] == 'CASA DOAN', ['PDV GPS', 'Ecommerce']] = ['CASA DOAN', False]
df.loc[df['Grupo economico'] == 'GRUPO GLOBAL', ['PDV GPS', 'Ecommerce']] = ['FARMAQUEN', False]
df.loc[df['Grupo economico'] == 'FARMASUR S.R.L.', ['PDV GPS', 'Ecommerce']] = ['FARMASUR S.R.L.', False]
df.loc[df['Grupo economico'] == 'HERCOM S.A.S', ['PDV GPS', 'Ecommerce']] = ['HERCOM S.A.S', False]
df.loc[df['Grupo economico'] == 'GRUPO RIADIGOS', ['PDV GPS', 'Ecommerce']] = ['GRUPO RIÃDIGOS', False]

In [162]:
# df.loc[df['Grupo economico'] == 20, 'Punto de Venta'] = df.loc[df['Grupo economico'] == 20, 'Punto de Venta'].str.replace('.1', '')


# Manejo de Errores

## Corrección de EAN´s

In [163]:
ean_errors= df[df['Ean sist']==False]
ean_errors_unique = ean_errors[["Ean", "Marca", "Desc Prod","Grupo economico"]].drop_duplicates().values.tolist()
ean_errors_unique=pd.DataFrame(ean_errors_unique,columns=["Ean", "Marca", "Desc Prod","Grupo economico"])
ean_errors_unique

,Ean,Marca,Desc Prod,Grupo economico


## Correcion de PDV´s

In [164]:
gps_errors=df[df['_merge']=='left_only']
gps_errors_unique = gps_errors[["Grupo economico", "Grupo economico","Punto de Venta","Codigo cliente", "PDV GPS"]].drop_duplicates().values.tolist()
gps_errors_unique= pd.DataFrame(gps_errors_unique,columns=["Grupo economico", "Grupo economico","Punto de Venta","Codigo cliente", "PDV GPS"])
gps_errors_unique

,Grupo economico,Grupo economico,Punto de Venta,Codigo cliente,PDV GPS
0,ASOPROFARMA C.P.L.,ASOPROFARMA C.P.L.,nan,NaN,ASOPROFARMA C.P.L.
1,CASA DOAN,CASA DOAN,nan,NaN,CASA DOAN
2,COOP.FARMACEUTICA MENDOZA LTDA.,COOP.FARMACEUTICA MENDOZA LTDA.,nan,NaN,COOP.FARMACEUTICA MENDOZA LTDA.
3,COFARSUR S.A.,COFARSUR S.A.,nan,NaN,COFARSUR S.A.
4,DROGUERIA SUR S.A.,DROGUERIA SUR S.A.,nan,NaN,DROGUERIA SUR S.A.
5,FARMAX SRL,FARMAX SRL,nan,NaN,FARMAX SRL
6,FIRST LABEL SRL,FIRST LABEL SRL,nan,NaN,FIRST LABEL SRL.
7,GRUPO GLOBAL,GRUPO GLOBAL,nan,NaN,FARMAQUEN
8,NIPPON,NIPPON,nan,NaN,Nippon Nippon
9,PERFUMERIAS ALDAZ,PERFUMERIAS ALDAZ,nan,NaN,PERFUMERIAS ALDAZ


## Join de Nuevos errores de EAN´s con base historica

In [165]:
ean_output_mapeo = pd.concat([ean_errors_unique, ean_historico], ignore_index=True)
ean_output_mapeo['Ean Correcto'] = ean_output_mapeo['Ean Correcto'].fillna(0)
df_conteo = ean_output_mapeo.groupby("Grupo economico").size().reset_index(name="Ean por mapear")
ean_output_mapeo=ean_errors_unique
ean_output_mapeo["Ean Correcto"]=0
ean_output_mapeo = pd.concat([ean_output_mapeo, ean_historico])

## Descargar archivo de salida EAN_MApeo

In [166]:
file_path = r'C:\Users\ArnaldoCarrasco\Documents\SO SCRIPT\Arvhivos de salida\EAN_Mapeo.xlsx'
ean_output_mapeo.to_excel(file_path, index=False)

# Filtrado de Grupos Economicos sin errores, y Validaciones de datos

In [167]:
df

,Ean,Desc Prod,Marca,Punto de Venta,Cantidad,Importe Neto,Stock,origen,Fecha,Grupo economico,Tipo de Venta,Ean sist,PDV GPS,Codigo cliente,Grupo economico Final,Ecommerce,_merge
0,3616305447197,ADIDAS CHILL ZONE EDP 100,NaN,nan,0.0,0.0,22,1012_202507,2025-07-01,ASOPROFARMA C.P.L.,Sell out,True,ASOPROFARMA C.P.L.,NaN,NaN,False,left_only
1,3616305447067,ADIDAS ENERGY DRI EDP 100,NaN,nan,3.0,0.0,11,1012_202507,2025-07-01,ASOPROFARMA C.P.L.,Sell out,True,ASOPROFARMA C.P.L.,NaN,NaN,False,left_only
2,3616305447050,ADIDAS FULL RECHAR EDP100,NaN,nan,1.0,0.0,17,1012_202507,2025-07-01,ASOPROFARMA C.P.L.,Sell out,True,ASOPROFARMA C.P.L.,NaN,NaN,False,left_only
3,3616305447166,ADIDAS GET COMFY EDP 100,NaN,nan,1.0,0.0,16,1012_202507,2025-07-01,ASOPROFARMA C.P.L.,Sell out,True,ASOPROFARMA C.P.L.,NaN,NaN,False,left_only
4,3616305447159,ADIDAS HAPPY FEELS EDP100,NaN,nan,2.0,0.0,23,1012_202507,2025-07-01,ASOPROFARMA C.P.L.,Sell out,True,ASOPROFARMA C.P.L.,NaN,NaN,False,left_only
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
484214,3616306361805,RIM- BLUR IT OUT KIND & FREE,NaN,nan,1.0,13801.0,0,22_202507,2025-07-01,TIENDA LOS GALLEGOS,Sell out,True,TIENDA LOS GALLEGOS,NaN,NaN,False,left_only
484215,3616303995683,RIM- KIND & FREE CLEAN BROWN WAX,NaN,nan,1.0,7272.0,0,22_202507,2025-07-01,TIENDA LOS GALLEGOS,Sell out,True,TIENDA LOS GALLEGOS,NaN,NaN,False,left_only
484216,3616303995683,RIM- KIND & FREE CLEAN BROWN WAX,NaN,nan,1.0,7272.0,0,22_202507,2025-07-01,TIENDA LOS GALLEGOS,Sell out,True,TIENDA LOS GALLEGOS,NaN,NaN,False,left_only
484217,3616303995683,RIM- KIND & FREE CLEAN BROWN WAX,NaN,nan,1.0,7272.0,0,22_202507,2025-07-01,TIENDA LOS GALLEGOS,Sell out,True,TIENDA LOS GALLEGOS,NaN,NaN,False,left_only


In [168]:
tabla_estadisticas = df.groupby('Grupo economico').agg(
    Ean_erroneos=('Ean sist', lambda x: (x == False).any())
).reset_index()
tabla_estadisticas

,Grupo economico,Ean_erroneos
0,ADM FRAGANCIAS S.R.L.,False
1,ASOPROFARMA C.P.L.,False
2,CASA DOAN,False
3,COFARSUR S.A.,False
4,COOP.FARMACEUTICA MENDOZA LTDA.,False
5,"CORTASSA ALDO, CORTASSA DIEGO,CORTASSA D",False
6,DROGUERIA SUR S.A.,False
7,DUTY PAID S.A.,False
8,FARMACIA DEL PUEBLO S.C.S.,False
9,FARMACITY,False


In [169]:
grupos_con_falso = tabla_estadisticas.loc[tabla_estadisticas['Ean_erroneos'], 'Grupo economico'].tolist()
grupos_con_falso

[]

In [170]:
grupo_ok = df[~df['Grupo economico'].isin(grupos_con_falso)]
grupo_ok

,Ean,Desc Prod,Marca,Punto de Venta,Cantidad,Importe Neto,Stock,origen,Fecha,Grupo economico,Tipo de Venta,Ean sist,PDV GPS,Codigo cliente,Grupo economico Final,Ecommerce,_merge
0,3616305447197,ADIDAS CHILL ZONE EDP 100,NaN,nan,0.0,0.0,22,1012_202507,2025-07-01,ASOPROFARMA C.P.L.,Sell out,True,ASOPROFARMA C.P.L.,NaN,NaN,False,left_only
1,3616305447067,ADIDAS ENERGY DRI EDP 100,NaN,nan,3.0,0.0,11,1012_202507,2025-07-01,ASOPROFARMA C.P.L.,Sell out,True,ASOPROFARMA C.P.L.,NaN,NaN,False,left_only
2,3616305447050,ADIDAS FULL RECHAR EDP100,NaN,nan,1.0,0.0,17,1012_202507,2025-07-01,ASOPROFARMA C.P.L.,Sell out,True,ASOPROFARMA C.P.L.,NaN,NaN,False,left_only
3,3616305447166,ADIDAS GET COMFY EDP 100,NaN,nan,1.0,0.0,16,1012_202507,2025-07-01,ASOPROFARMA C.P.L.,Sell out,True,ASOPROFARMA C.P.L.,NaN,NaN,False,left_only
4,3616305447159,ADIDAS HAPPY FEELS EDP100,NaN,nan,2.0,0.0,23,1012_202507,2025-07-01,ASOPROFARMA C.P.L.,Sell out,True,ASOPROFARMA C.P.L.,NaN,NaN,False,left_only
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
484214,3616306361805,RIM- BLUR IT OUT KIND & FREE,NaN,nan,1.0,13801.0,0,22_202507,2025-07-01,TIENDA LOS GALLEGOS,Sell out,True,TIENDA LOS GALLEGOS,NaN,NaN,False,left_only
484215,3616303995683,RIM- KIND & FREE CLEAN BROWN WAX,NaN,nan,1.0,7272.0,0,22_202507,2025-07-01,TIENDA LOS GALLEGOS,Sell out,True,TIENDA LOS GALLEGOS,NaN,NaN,False,left_only
484216,3616303995683,RIM- KIND & FREE CLEAN BROWN WAX,NaN,nan,1.0,7272.0,0,22_202507,2025-07-01,TIENDA LOS GALLEGOS,Sell out,True,TIENDA LOS GALLEGOS,NaN,NaN,False,left_only
484217,3616303995683,RIM- KIND & FREE CLEAN BROWN WAX,NaN,nan,1.0,7272.0,0,22_202507,2025-07-01,TIENDA LOS GALLEGOS,Sell out,True,TIENDA LOS GALLEGOS,NaN,NaN,False,left_only


# Generacion de Archivo Final

## Ultimas Transformaciones

In [171]:
grupo_ok=df

In [172]:
grupo_ok.drop(columns=[
    'origen',
    'Punto de Venta',
    'Ean sist',
    '_merge',
    'Grupo economico Final',
    'Codigo cliente'
], inplace=True)

grupo_ok.rename(columns={
    'PDV GPS': 'Punto de Venta',
    'Ecommerce': 'Columna1'
}, inplace=True)
grupo_ok

,Ean,Desc Prod,Marca,Cantidad,Importe Neto,Stock,Fecha,Grupo economico,Tipo de Venta,Punto de Venta,Columna1
0,3616305447197,ADIDAS CHILL ZONE EDP 100,NaN,0.0,0.0,22,2025-07-01,ASOPROFARMA C.P.L.,Sell out,ASOPROFARMA C.P.L.,False
1,3616305447067,ADIDAS ENERGY DRI EDP 100,NaN,3.0,0.0,11,2025-07-01,ASOPROFARMA C.P.L.,Sell out,ASOPROFARMA C.P.L.,False
2,3616305447050,ADIDAS FULL RECHAR EDP100,NaN,1.0,0.0,17,2025-07-01,ASOPROFARMA C.P.L.,Sell out,ASOPROFARMA C.P.L.,False
3,3616305447166,ADIDAS GET COMFY EDP 100,NaN,1.0,0.0,16,2025-07-01,ASOPROFARMA C.P.L.,Sell out,ASOPROFARMA C.P.L.,False
4,3616305447159,ADIDAS HAPPY FEELS EDP100,NaN,2.0,0.0,23,2025-07-01,ASOPROFARMA C.P.L.,Sell out,ASOPROFARMA C.P.L.,False
...,...,...,...,...,...,...,...,...,...,...,...
484214,3616306361805,RIM- BLUR IT OUT KIND & FREE,NaN,1.0,13801.0,0,2025-07-01,TIENDA LOS GALLEGOS,Sell out,TIENDA LOS GALLEGOS,False
484215,3616303995683,RIM- KIND & FREE CLEAN BROWN WAX,NaN,1.0,7272.0,0,2025-07-01,TIENDA LOS GALLEGOS,Sell out,TIENDA LOS GALLEGOS,False
484216,3616303995683,RIM- KIND & FREE CLEAN BROWN WAX,NaN,1.0,7272.0,0,2025-07-01,TIENDA LOS GALLEGOS,Sell out,TIENDA LOS GALLEGOS,False
484217,3616303995683,RIM- KIND & FREE CLEAN BROWN WAX,NaN,1.0,7272.0,0,2025-07-01,TIENDA LOS GALLEGOS,Sell out,TIENDA LOS GALLEGOS,False


In [173]:
grupo_ok = df[~df['Grupo economico'].isin(grupos_con_falso)]
# grupo_ok = grupo_ok.drop(columns=["Desc Prod", "Grupo economico", "Marca", "Punto de Venta", "LRC", "Ean sist", "Ean LRC", "Codigo cliente", "_merge"])
grupo_ok['Importe Neto'] = grupo_ok['Importe Neto'].replace('nan', 0)
grupo_ok["Cantidad"] = grupo_ok["Cantidad"].replace([np.inf, -np.inf], np.nan).fillna(0).astype(int)
grupo_ok["Cantidad"] = grupo_ok["Cantidad"].astype(int)
# grupo_ok.rename(columns={
#     "PDV GPS": "Punto de Venta",
#     "Grupo economico Final": "Grupo economico",
#     "Ecommerce": "Columna1"
# }, inplace=True)
grupo_ok["Columna1"] = grupo_ok["Columna1"].replace({True: "Si", False: "No"})
grupo_ok['Importe Neto']=grupo_ok['Importe Neto'].fillna(0)
grupo_ok['Importe Neto']=grupo_ok['Importe Neto'].astype(int)
grupo_ok["Fecha"] = pd.to_datetime(grupo_ok["Fecha"], errors='coerce', dayfirst=True).dt.date
grupo_ok

,Ean,Desc Prod,Marca,Cantidad,Importe Neto,Stock,Fecha,Grupo economico,Tipo de Venta,Punto de Venta,Columna1
0,3616305447197,ADIDAS CHILL ZONE EDP 100,NaN,0,0,22,2025-07-01,ASOPROFARMA C.P.L.,Sell out,ASOPROFARMA C.P.L.,No
1,3616305447067,ADIDAS ENERGY DRI EDP 100,NaN,3,0,11,2025-07-01,ASOPROFARMA C.P.L.,Sell out,ASOPROFARMA C.P.L.,No
2,3616305447050,ADIDAS FULL RECHAR EDP100,NaN,1,0,17,2025-07-01,ASOPROFARMA C.P.L.,Sell out,ASOPROFARMA C.P.L.,No
3,3616305447166,ADIDAS GET COMFY EDP 100,NaN,1,0,16,2025-07-01,ASOPROFARMA C.P.L.,Sell out,ASOPROFARMA C.P.L.,No
4,3616305447159,ADIDAS HAPPY FEELS EDP100,NaN,2,0,23,2025-07-01,ASOPROFARMA C.P.L.,Sell out,ASOPROFARMA C.P.L.,No
...,...,...,...,...,...,...,...,...,...,...,...
484214,3616306361805,RIM- BLUR IT OUT KIND & FREE,NaN,1,13801,0,2025-07-01,TIENDA LOS GALLEGOS,Sell out,TIENDA LOS GALLEGOS,No
484215,3616303995683,RIM- KIND & FREE CLEAN BROWN WAX,NaN,1,7272,0,2025-07-01,TIENDA LOS GALLEGOS,Sell out,TIENDA LOS GALLEGOS,No
484216,3616303995683,RIM- KIND & FREE CLEAN BROWN WAX,NaN,1,7272,0,2025-07-01,TIENDA LOS GALLEGOS,Sell out,TIENDA LOS GALLEGOS,No
484217,3616303995683,RIM- KIND & FREE CLEAN BROWN WAX,NaN,1,7272,0,2025-07-01,TIENDA LOS GALLEGOS,Sell out,TIENDA LOS GALLEGOS,No


## Generacion de archivo de salida

In [174]:
grupo_ok['Ean'] = grupo_ok['Ean'].astype('Int64').astype(str)


In [175]:
file_path = r'C:\Users\ArnaldoCarrasco\Documents\SO SCRIPT\Arvhivos de salida\SO_NEW_Final.csv'
grupo_ok.to_csv(file_path, index=False)

In [176]:
grupo_ok['Ean'] = grupo_ok['Ean'].astype('Int64').astype(str)


# Validacion de Datos Archivo de Salida vs Consolidado

In [177]:
# Copias de seguridad
status = grupo_ok.copy()
status2 = df_historico.copy()

# ---- TRABAJO SOBRE STATUS (cantidad salida) ----
# Extraer mes y año
status['Fecha'] = pd.to_datetime(status['Fecha'], errors='coerce', dayfirst=True)
status['Mes'] = status['Fecha'].dt.month
status['Año'] = status['Fecha'].dt.year

# Agrupar por año, mes y grupo económico
tabla_salida = (
    status
    .groupby(['Año', 'Mes', 'Grupo economico'], as_index=False)
    .agg(Total_Cantidad_salida=('Cantidad', 'sum'))
)

# Asegurar tipos correctos
tabla_salida['Año'] = tabla_salida['Año'].astype(int)
tabla_salida['Mes'] = tabla_salida['Mes'].astype(int)
tabla_salida['Grupo economico'] = tabla_salida['Grupo economico'].astype(str)
tabla_salida['Total_Cantidad_salida'] = tabla_salida['Total_Cantidad_salida'].astype(np.int64)

# ---- TRABAJO SOBRE STATUS2 (cantidad original) ----
# Completar fechas nulas
status2.loc[status2['Fecha'].isna(), 'Fecha'] = pd.to_datetime('1/3/2025', format='%d/%m/%Y')

# Convertir fecha correctamente
status2['Fecha'] = pd.to_datetime(status2['Fecha'], errors='coerce', dayfirst=True)


# Extraer mes y año
status2['Mes'] = status2['Fecha'].dt.month
status2['Año'] = status2['Fecha'].dt.year

# Agrupar
tabla_original = (
    status2
    .groupby(['Año', 'Mes', 'Grupo economico'], as_index=False)
    .agg(Total_Cantidad_original=('Cantidad', 'sum'))
)

# # Merge con gps_cliente para corregir nombre de grupo económico
# tabla_original = pd.merge(
#     tabla_original,
#     gps_cliente[['ID_CLIENTE', 'Nombre Correcto']],
#     left_on='Grupo economico',
#     right_on='ID_CLIENTE',
#     how='left'
# )

# Limpiar columnas
# tabla_original.drop(columns=['ID_CLIENTE', 'Grupo economico'], inplace=True)
# tabla_original.rename(columns={'Nombre Correcto': 'Grupo economico'}, inplace=True)

# Asegurar tipos correctos
tabla_original['Año'] = tabla_original['Año'].astype(int)
tabla_original['Mes'] = tabla_original['Mes'].astype(int)
tabla_original['Grupo economico'] = tabla_original['Grupo economico'].astype(str)

# Reordenar columnas
tabla_original = tabla_original[['Año', 'Mes', 'Grupo economico', 'Total_Cantidad_original']]

# ---- MERGE FINAL ----
df_comparado = pd.merge(
    tabla_original,
    tabla_salida,
    on=['Año', 'Mes', 'Grupo economico'],
    how='outer'
)
df_comparado['Diferencia'] = (
        df_comparado['Total_Cantidad_salida'].fillna(0) -
        df_comparado['Total_Cantidad_original'].fillna(0)
)
# df_comparado['Porcentaje'] = (
#         (df_comparado['Total_Cantidad_original'].fillna(0)/
#         df_comparado['Total_Cantidad_salida'].fillna(0))
# )
df_comparado['Porcentaje'] = (
        df_comparado['Total_Cantidad_salida'].fillna(0) /
        df_comparado['Total_Cantidad_original'].replace(0, pd.NA)
)*100
df_comparado = df_comparado.sort_values(
    by=['Año', 'Mes', 'Grupo economico'],
    ascending=[True, True, True]
)

# Mostrar resultado
df_comparado    


,Año,Mes,Grupo economico,Total_Cantidad_original,Total_Cantidad_salida,Diferencia,Porcentaje
0,2024,12,FARMACITY,208002.000000,207996.0,-6.000000,99.997115
1,2025,1,FARMACITY,99884.000000,99881.0,-3.000000,99.996997
2,2025,2,FARMACITY,96892.000000,96892.0,0.000000,100.000000
3,2025,3,FARMACITY,165653.000000,165650.0,-3.000000,99.998189
4,2025,4,FARMACITY,109136.000000,109132.0,-4.000000,99.996335
5,2025,5,FARMACITY,146444.000000,146440.0,-4.000000,99.997269
6,2025,6,COOP.FARMACEUTICA MENDOZA LTDA.,400.000000,NaN,-400.000000,0.000000
7,2025,6,FARMACITY,160848.000000,160845.0,-3.000000,99.998135
8,2025,7,ADM FRAGANCIAS S.R.L.,24511.000000,NaN,-24511.000000,0.000000
9,2025,7,ASOPROFARMA C.P.L.,977.000000,NaN,-977.000000,0.000000


In [178]:
print(df_comparado[['Total_Cantidad_original', 'Total_Cantidad_salida']])


    Total_Cantidad_original  Total_Cantidad_salida
0             208002.000000               207996.0
1              99884.000000                99881.0
2              96892.000000                96892.0
3             165653.000000               165650.0
4             109136.000000               109132.0
5             146444.000000               146440.0
6                400.000000                    NaN
7             160848.000000               160845.0
8              24511.000000                    NaN
9                977.000000                    NaN
10              1574.000000                 1574.0
11                55.000000                   55.0
12              3958.000000                 3950.0
13              6036.000000                    NaN
14               883.000000                  883.0
15               206.000000                  115.0
16            128342.000000               128342.0
17              3838.000000                    NaN
18               748.000000    